# Kaldu Reviews Analytics (Public/Non-confidential Data)
**Notebook**: End-to-end EDA, sentiment analysis, and topic cues.


In [ ]:
import pandas as pd, numpy as np, re, string, json
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.cluster import KMeans

# Load dataset
df = pd.read_excel("../data/kaldu-cleaning.xlsx", sheet_name=0).copy()
df = df.drop_duplicates(subset=["author", "timestamp", "text"]).reset_index(drop=True)

def try_parse_datetime(x):
    try:
        return pd.to_datetime(x)
    except Exception:
        return pd.NaT
df["timestamp_parsed"] = df["timestamp"].apply(try_parse_datetime)

df["text"] = df["text"].fillna("")
df["owner_reply"] = df["owner_reply"].fillna("")
df["has_owner_reply"] = df["owner_reply"].str.strip().ne("")
print(df.head())

## Sentiment Analysis
We use VADER if available, otherwise a tiny Indonesian/English lexicon fallback (illustrative).

In [ ]:
use_vader = False
scores = None
try:
    from nltk.sentiment import SentimentIntensityAnalyzer
    import nltk
    try:
        _ = nltk.data.find('sentiment/vader_lexicon.zip')
    except LookupError:
        nltk.download('vader_lexicon', quiet=True)
    sia = SentimentIntensityAnalyzer()
    use_vader = True
except Exception as e:
    use_vader = False

pos_words = set("""mantap enak lezat nikmat recommended cepat ramah bersih murah terbaik puas suka love great good awesome nice delicious tasty friendly clean cheap fast amazing fresh helpful nyaman cepat""".split())
neg_words = set("""buruk lama mahal tidak enak basi mengecewakan kotor lambat parah jelek pahit asin asam overprice overprized overpricey wait cold rude raw undercooked oily hambar""".split())

import re
def simple_sentiment_score(text):
    tokens = re.findall(r"\w+", str(text).lower())
    if not tokens:
        return 0.0
    score = sum(1 for t in tokens if t in pos_words) - sum(1 for t in tokens if t in neg_words)
    return score / max(len(tokens), 1)

if use_vader:
    df["sentiment"] = df["text"].apply(lambda t: sia.polarity_scores(str(t))["compound"])
else:
    df["sentiment"] = df["text"].apply(simple_sentiment_score)

def to_label(s):
    if s >= 0.2: 
        return "positive"
    elif s <= -0.2:
        return "negative"
    else:
        return "neutral"
df["sentiment_label"] = df["sentiment"].apply(to_label)
df["sentiment_label"].value_counts()

## N-grams and Topic Cues

In [ ]:
def normalize_text(t):
    t = str(t).lower()
    t = re.sub(r"http\S+", " ", t)
    import string
    t = t.translate(str.maketrans("", "", string.punctuation))
    t = re.sub(r"\d+", " ", t)
    t = re.sub(r"\s+", " ", t).strip()
    return t

df["text_norm"] = df["text"].apply(normalize_text)

stop_id = set("""yang dan di ke dari untuk pada dengan ini itu itu nya si nya nya lah kok dong deh kan juga kita kamu saya saya nya tidak bukan ya sudah aja biar agar kalau karena jadi saat oleh dalam mereka dia nya para para para para""".split())
stop_en = set("""the a an and or of in on at to for from as is are was were be been being by with this that it its you your we our they their them he she his her i me my mine""".split())
stop = list(stop_id | stop_en)

vectorizer = CountVectorizer(min_df=3, max_df=0.9, ngram_range=(1,2), stop_words=stop)
X_counts = vectorizer.fit_transform(df["text_norm"])
vocab = vectorizer.get_feature_names_out()
sum_counts = X_counts.sum(axis=0).A1
top_idx = sum_counts.argsort()[::-1][:20]
top_terms = [(vocab[i], int(sum_counts[i])) for i in top_idx]
top_terms[:10]

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Reviews per month
if df["timestamp_parsed"].notna().any():
    df["month"] = df["timestamp_parsed"].dt.to_period("M").astype(str)
else:
    df["month"] = "unknown"

reviews_per_month = df.groupby("month").size().reset_index(name="reviews")
reviews_per_month = reviews_per_month.sort_values("month")

plt.figure()
plt.plot(reviews_per_month["month"], reviews_per_month["reviews"], marker="o")
plt.title("Reviews per Month")
plt.xlabel("Month")
plt.ylabel("Count")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()

# Sentiment distribution
sentiment_share = df["sentiment_label"].value_counts().reindex(["positive","neutral","negative"]).fillna(0).astype(int)
plt.figure()
sentiment_share.plot(kind="bar")
plt.title("Sentiment Distribution")
plt.xlabel("Sentiment")
plt.ylabel("Count")
plt.tight_layout()
plt.show()

# Top terms bar
terms, counts = zip(*top_terms) if top_terms else ([], [])
plt.figure()
plt.barh(range(len(terms))[::-1], list(counts)[::-1])
plt.yticks(range(len(terms))[::-1], list(terms)[::-1])
plt.title("Top Terms / N-grams")
plt.xlabel("Frequency")
plt.tight_layout()
plt.show()

In [ ]:
# Simple topics via KMeans (unsupervised)
topics = []
if X_counts.shape[0] >= 20 and X_counts.shape[1] >= 10:
    k = 5 if X_counts.shape[0] > 100 else 3
    km = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels = km.fit_predict(X_counts)
    df["topic"] = labels
    centers = km.cluster_centers_
    for i in range(k):
        top_terms_idx = centers[i].argsort()[::-1][:8]
        topics.append({"topic_id": i, "top_terms": [vocab[j] for j in top_terms_idx], "size": int((labels == i).sum())})
topics

## Owner Reply Rate

In [ ]:
owner_reply_rate = df["has_owner_reply"].mean() if len(df)>0 else 0.0
owner_reply_rate